In [1]:
import cPickle as pickle
import os, sys, random
sys.path.append('/Users/matt.meng/dev/seq2seq_model')

In [22]:
import pandas as pd
from data_preprocess import tokenize_title_column, convert_text_JSON_to_csv, process_title_column, \
process_title_column_by_spacy, create_selected_vocab_dict, process_title_with_token_dict, create_crambled_training

### the title processing steps

In [3]:
data_path = '/Users/matt.meng/Downloads'
file_name = 'insights_selected_articles.JSON'
meta_data_file_name = 'meta_title_data.csv'
delimiter = '\t\t'

In [4]:
convert_text_JSON_to_csv(os.path.join(data_path, file_name),
                         os.path.join(data_path, meta_data_file_name),
                         delimiter)

finished processing 40000 rows using 3.25 seconds
finished processing 80000 rows using 6.45 seconds
finished processing 120000 rows using 10.61 seconds
finished processing 160000 rows using 14.04 seconds
finished processing 200000 rows using 17.10 seconds
finished processing 240000 rows using 20.19 seconds
finished processing 280000 rows using 23.34 seconds
finished processing 320000 rows using 26.52 seconds
finished processing 360000 rows using 29.65 seconds
finished processing 400000 rows using 32.83 seconds
finished processing 440000 rows using 35.94 seconds
finished processing 480000 rows using 39.09 seconds
finished processing 520000 rows using 42.27 seconds
finished processing 560000 rows using 45.32 seconds
finished processing 600000 rows using 48.45 seconds
finished processing 640000 rows using 51.52 seconds
finished processing 680000 rows using 54.64 seconds
finished processing 720000 rows using 57.70 seconds
finished processing 760000 rows using 60.79 seconds
finished process

In [5]:
data = pd.read_csv(os.path.join(data_path, meta_data_file_name), index_col='url', delimiter=delimiter, encoding='utf-8')


/Users/matt.meng/.virtualenvs/kaggle/lib/python2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [6]:
print data.shape
data.head()

(774000, 3)


,title,traffic,publisherId
url,,,
https://www.cbsnews.com/videos/adults-less-likely-to-use-seat-belts-in-hired-cars-survey-finds/,Adults less likely to use seat belts in hired ...,133,1038583
https://www.cbsnews.com/news/palestinian-gunman-kills-3-israelis-in-settlement-near-jerusalem/,Palestinian gunman kills 3 Israelis in settlem...,1023,1038583
https://www.cbsnews.com/videos/dick-cavett-rebuilds-a-montauk-treasure/,Dick Cavett rebuilds a Montauk treasure,94,1038583
https://www.cbsnews.com/news/college-isnt-paying-off-for-many-minorities/,College isn't paying off for many minorities,156,1038583
https://www.cbsnews.com/news/fast-food-linked-to-child-obesity/,Fast Food Linked To Child Obesity,74,1038583


In [16]:
data.sort_values(['traffic'], ascending=False, inplace=True)

In [17]:
data.head()

,title,traffic,publisherId
url,,,
https://weather.com/storms/hurricane/news/hurricane-irma-bahamas-florida-georgia-carolinas-forecast,Hurricane Irma on Its Way to the Bahamas as a ...,11508824,1187
https://weather.com/storms/hurricane/news/hurricane-irma-united-states-hurricane-warning-puerto-rico-leeward-islands-0,Hurricane Irma on Its Way to the Bahamas as a ...,9787772,1187
http://newarena.com/nfl/ex-nfl-player-who-wanted-son-dead-17-years-ago-is-in-for-a-shocking-surprise/,Ex-NFL Player Who Wanted Son Dead 17 Years Ago...,9016645,1023235
https://weather.com/storms/hurricane/news/tropical-storm-hurricane-jose-forecast-atlantic,Tropical Storm Jose Strengthens in the Atlanti...,4401042,1187
https://weather.com/storms/hurricane/news/hurricane-irma-florida-georgia-south-carolina-north-carolina-forecast-path,Hurricane Irma's Track Forecast: Here's What t...,3367893,1187


In [18]:
sum(data.index.duplicated())

28219

In [19]:
unique_data = data[~data.index.duplicated(keep='first')]


In [20]:
print unique_data.shape


(745781, 3)


In [21]:
processed_column_name = 'processed_title'
pageView_column_name = 'traffic'
filtered_data = tokenize_title_column(unique_data, processed_column_name, pageView_column_name)

/Users/matt.meng/dev/seq2seq_model/data_preprocess.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  filtered_data(DataFrame): a selected Pandas DataFrame


finish the tokenization...


In [29]:
print filtered_data.shape
filtered_data.head()

(681539, 5)


,title,traffic,publisherId,title_word_counts,processed_title
url,,,,,
http://www.msn.com/en-us/news/breakingnews/live-coverage-from-cbs-news/ar-BBmYvYY,Live Coverage from CBS News,1671365,1023406,5,live coverage from cbs news
http://clark.com/personal-finance-credit/credit-freeze-and-thaw-guide/,Credit freeze and thaw guide,1550744,1016433,5,credit freeze and thaw guide
http://www.nbcmiami.com/news/local/Watch-Live-NBC-6-News-204316851.html,WATCH LIVE: NBC 6 News,802043,1030944,5,watch live nbc ## news
https://weather.com/tv/the-weather-channel-live/video/watch-the-weather-channel-live,Watch The Weather Channel Live,569573,1187,5,watch the weather channel live
http://www.obsev.com/galleries/bizarre-thrift-store-finds/445685,Insane Luxury Dog Houses | Obsev,443215,1061393,5,insane luxury dog houses obsev


In [35]:
titile_dict = {}
for title in filtered_data['processed_title']:
    titile_dict[title] = titile_dict.get(title, 0) + 1

In [36]:
sorted_titles = sorted(titile_dict.items(), key=lambda x: x[1], reverse=True)


In [38]:
sorted_titles[:100]

[('former tulsa principal pleads guilty to child porn charge news## com',
  1611),
 ('espn updates fpi rankings after week ##', 512),
 ('big brother video episode ## cbs com', 349),
 ('week ## college football bowl projections', 121),
 ('ap top entertainment news at ## ## p m edt', 113),
 ('## things to know for today', 104),
 ('ap top international news at ## ## p m edt', 103),
 ('week ## amway coaches poll released', 99),
 ('steely dan co founder guitarist walter becker dies at ##', 99),
 ('see the entire kennedy family', 93),
 ('the girlfriends of college football stars', 92),
 ('t mobile galaxy note ## pre order thread page ## android forums at androidcentral com',
  84),
 ('palace announces prince william kate expecting third child', 82),
 ('fox ## phoenix breaking news local headlines weather traffic and more ksaz',
  78),
 ('price is right made history and everyone lost their minds', 78),
 ('nfl players irate over president trump s comments', 77),
 ('federal government notifies 

In [27]:
processed_column_name = 'processed_title'
pageView_column_name = 'traffic'

all_titles, vocab_dict = process_title_column(filtered_data, processed_column_name, pageView_column_name)


total 76645 words are tokenized from 451787 titles using 5.55 second


In [26]:
'''
data['publisherId'] = data['publisherId'].astype(int).astype(str)
valid_publisher_ids = ['1001082', '1023406', '1003264', '1040522', '782', '1006541',
                           '1168', '1038583', '1021516', '580', '1020689', '1031851', '1001264',
                           '1039208', '1054980', '1018671', '1031841', '1031842', '1031852',
                           '1008941', '1003764', '1068057', '1038711', '1002628', '1031853',
                           '1021578', '1043813', '1010748', '1040526', '1005092', '612',
                           '1003870', '1001156', '1012083', '1017946', '1041479', '1027016',
                           '1010488', '1017947', '1010497', '1038582', '1045821', '1020968',
                           '1037842', '1029984', '723', '196', '1030941']

filtered_data = data.loc[data['publisherId'].isin(valid_publisher_ids), :]
unique_filtered_data = filtered_data[~filtered_data.index.duplicated(keep='first')]
print unique_filtered_data.shape

processed_column_name = 'processed_title'
pageView_column_name = 'traffic'
all_titles, vocab_dict = process_title_column_by_spacy(filtered_data, 'processed_title', 'traffic', skip_stop_words=True)
'''


"\ndata['publisherId'] = data['publisherId'].astype(int).astype(str)\nvalid_publisher_ids = ['1001082', '1023406', '1003264', '1040522', '782', '1006541',\n                           '1168', '1038583', '1021516', '580', '1020689', '1031851', '1001264',\n                           '1039208', '1054980', '1018671', '1031841', '1031842', '1031852',\n                           '1008941', '1003764', '1068057', '1038711', '1002628', '1031853',\n                           '1021578', '1043813', '1010748', '1040526', '1005092', '612',\n                           '1003870', '1001156', '1012083', '1017946', '1041479', '1027016',\n                           '1010488', '1017947', '1010497', '1038582', '1045821', '1020968',\n                           '1037842', '1029984', '723', '196', '1030941']\n\nfiltered_data = data.loc[data['publisherId'].isin(valid_publisher_ids), :]\nunique_filtered_data = filtered_data[~filtered_data.index.duplicated(keep='first')]\nprint unique_filtered_data.shape\n\nproces

#### check the `vocab_ditc` content

In [30]:
sorted_pairs = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)

In [31]:
sorted_pairs

[('##', 125613),
 ('to', 112721),
 ('in', 100492),
 ('the', 81088),
 ('of', 64349),
 ('for', 61409),
 ('on', 41230),
 ('and', 32707),
 ('with', 30320),
 ('at', 28756),
 ('is', 24654),
 ('after', 23301),
 ('new', 19686),
 ('from', 18842),
 ('man', 17382),
 ('hurricane', 16300),
 ('trump', 15641),
 ('you', 15607),
 ('how', 15593),
 ('not', 14779),
 ('police', 14624),
 ('irma', 13439),
 ('are', 12591),
 ('as', 12497),
 ('by', 12207),
 ('your', 11069),
 ('this', 10954),
 ('harvey', 10918),
 ('about', 10288),
 ('will', 10272),
 ('state', 10157),
 ('what', 9954),
 ('football', 9952),
 ('it', 9354),
 ('be', 9250),
 ('out', 9193),
 ('over', 9095),
 ('up', 9021),
 ('that', 8953),
 ('woman', 8380),
 ('school', 8133),
 ('says', 8102),
 ('year', 8065),
 ('video', 7934),
 ('home', 7399),
 ('county', 7355),
 ('more', 7092),
 ('who', 7076),
 ('game', 7053),
 ('florida', 6871),
 ('week', 6858),
 ('why', 6705),
 ('watch', 6645),
 ('his', 6587),
 ('first', 6332),
 ('old', 6249),
 ('can', 6236),
 ('has',

### create the `token_dict` and the scrambled data

In [33]:
_PAD = b"_PAD"
_GO = b"_GO"
_EOS = b"_EOS"
_UNK = b"_UNK"
_START_VOCAB = [_PAD, _GO, _EOS, _UNK]

TOKEN_DICT = {}
REVERSE_TOKEN_DICT = {}
for i in xrange(len(_START_VOCAB)):
    TOKEN_DICT[_START_VOCAB[i]] = i
    REVERSE_TOKEN_DICT[i] = _START_VOCAB[i]

In [34]:
UKN_index = len(TOKEN_DICT) - 1
token_dict, reverse_token_dict = create_selected_vocab_dict(vocab_dict, UKN_index, token_freq_threshold=4)
selected_content = process_title_with_token_dict(all_titles, token_dict, reverse_token_dict, UKN_index, UKN_frac_threshold=0.2)
processed_content = create_crambled_training(selected_content)

total 33779 unique tokens are included in the token dictionary...
total 424504 titles are included...
finish generating scrambled titles, ignore 0 titles..


### load the processed pickle and check the content

In [ ]:
data_path = '/Users/matt.meng'
#pickle_file = 'processed_titles_data.pkl'
pickle_file = 'lemmanized_no_stop_words_CBOW_data.pkl'
with open(os.path.join(data_path, pickle_file), 'r') as input_file:
     data = pickle.load(input_file)

In [ ]:
data.keys()

In [ ]:
training_list = data['training_list']
target_list = data['target_list']

In [ ]:
print len(training_list)

In [ ]:
training_list[:4]

In [ ]:
titles = data['titles']
token_dict = data['token_dict']
reverse_token_dict = data['reverse_token_dict']

In [ ]:
token_dict.keys()[:20]

In [ ]:
index = 50
map(reverse_token_dict.get, titles[index])

In [ ]:
'''
training_titles = data['training_titles']
target_titles = data['target_titles']

index = 20000
print training_titles[index]
print target_titles[index]
'''

### validate the DataGenerator with dual outputs

In [ ]:
from data_preprocess import TOKEN_DICT, _GO, _EOS
from data import DataGenerator, process_batch

In [ ]:
pickle_file = 'scramble_titles_data.pkl'
batch_size = 32
pickle_file_path = os.path.join(os.path.expanduser("~"), pickle_file)

## the dual_output test
#dataGen = DataGenerator(pickle_file_path)
dataGen = DataGenerator(pickle_file_path, True)
batches = dataGen.generate_sequence(batch_size)
training_batch, target_batch = next(batches)
  

In [ ]:
training_batch

In [ ]:
target_batch

In [ ]:
encoder_inputs_, encoder_inputs_length = process_batch([sequence + [TOKEN_DICT[_EOS]] for sequence in training_batch])
decoder_inputs_, decoder_inputs_length = process_batch([[TOKEN_DICT[_GO]] + sequence for sequence in target_batch])


In [ ]:
encoder_inputs_

In [ ]:
decoder_inputs_